In [1]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
## Keypoints using MP Holistic ##

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model -> make out detections
mp_drawing = mp.solutions.drawing_utils # drawing utilities -> draw them

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion BGR to RGB
    image.flags.writeable = False # Image no longer writeable
    results = model.process(image) # Make prediction (image is the frame)
    image.flags.writeable = True   # Image is now writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion back
    return image, results

In [6]:
def draw_landmarks(image, results):
    #connection maps
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) #connection for face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1 ),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1 ))

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=2))

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(thickness=2, circle_radius=2))

In [8]:
import cv2
#make sure we can access our webcam

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        #Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image) #img from webcam

        # Breaking gracefully by qlicking q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
#draw_landmarks(frame, results)
#plt.imshow(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

In [10]:
## Extract Keypoint Values ##

In [11]:
pose = [] # placeholder array
for res in results.pose_landmarks.landmark: #extract landmarks
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [12]:
test

array([ 4.39826697e-01,  3.21736264e+00, -3.42554510e-01,  6.47546025e-04])

In [13]:
def extract_keypoints(results):
    # Note: flatten for making it into one big array
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

In [14]:
result_test = extract_keypoints(results)

In [15]:
np.save('0', result_test)

In [16]:
np.load('0.npy')

array([ 0.62091738,  0.76117325, -1.12023079, ...,  0.5481143 ,
        0.98128015, -0.03300837])

In [17]:
## Setup Folders for Collection ##

In [18]:
# Var that holds path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') #were we will store the data

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
seq_length = 30

In [19]:
# Folders:
# Hello
## 0
## 1 .....
## 29
# Thanks
# I Love you

In [20]:
for action in actions:
    for seq in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(seq)))
        except:
            pass

In [21]:
## Collect Keypoint Values For Training and Testing ##

In [22]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW loop through actions
    for action in actions:
        # loop through sequences aka videos
        for seq in range(no_sequences):
            #loop through video length aka seq length
            for frame_num in range(seq_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Take a break between each video - NEW Wait logic
                if frame_num == 0:
                    cv2.putText(image, 'Starting collection', (120,200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action, seq), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action, seq), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

                # NEW Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(seq), str(frame_num))
                np.save(npy_path, keypoints)

                # Show to screen
                cv2.imshow('OpenCV Feed', image) #img from webcam

                # Breaking gracefully by clicking q
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## Preprocess Data and Create Labels and Features ##

In [24]:
from sklearn.model_selection import train_test_split #partion our data to train partition
from tensorflow.keras.utils import to_categorical

In [25]:
# label map
label_map = {label:num for num , label in enumerate(actions)}

In [26]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [27]:
sequences, labels = [], []
for action in actions:
    for seq in range(no_sequences):
        window = [] # All diff frames for particular seq
        for frame_num in range(seq_length):
            res = np.load(os.path.join(DATA_PATH, action, str(seq), "{}.npy".format(frame_num))) #looping through our frame
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(90, 30, 1662)

In [29]:
np.array(labels).shape

(90,)

In [30]:
x = np.array(sequences)

In [31]:
y = to_categorical(labels).astype(int) #category using binary 0,1

In [32]:
# Testing and training partition

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05) #0.05 -> 5% of our data

In [33]:
## Build and Train LSTM Neural Network ##

In [34]:
from tensorflow.keras.models import Sequential  # seq neural network
from tensorflow.keras.layers import LSTM, Dense  # temperal component -> action detection, Dense like layers
from tensorflow.keras.callbacks import TensorBoard # logging

In [35]:
#log directory

log_dir = os.path.join('Logs') # web app allows u to monitor ur training
tb_callback = TensorBoard(log_dir=log_dir)

In [36]:
model = Sequential() # Creates an instance of the Sequential class
# Sets
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu')) # false cuz next line is Dense
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # 3 neural network units

In [37]:
# this loss model u should use for multi cross classification model
# if binary cross model u use binary crossentropy

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [38]:
# train our model
# 2000 epochs too high here btw
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 2s 37ms/step - loss: 2.7146 - categorical_accuracy: 0.3882
Epoch 2/2000
3/3 [==============================] - 0s 39ms/step - loss: 2.3007 - categorical_accuracy: 0.2353
Epoch 3/2000
3/3 [==============================] - 0s 52ms/step - loss: 19.0977 - categorical_accuracy: 0.2941
Epoch 4/2000
3/3 [==============================] - 0s 53ms/step - loss: 13.9274 - categorical_accuracy: 0.2353
Epoch 5/2000
3/3 [==============================] - 0s 43ms/step - loss: 7.5148 - categorical_accuracy: 0.4118
Epoch 6/2000
3/3 [==============================] - 0s 45ms/step - loss: 10.8035 - categorical_accuracy: 0.2588
Epoch 7/2000
3/3 [==============================] - 0s 43ms/step - loss: 16.2688 - categorical_accuracy: 0.2941
Epoch 8/2000
3/3 [==============================] - 0s 43ms/step - loss: 7.2051 - categorical_accuracy: 0.3765
Epoch 9/2000
3/3 [==============================] - 0s 42ms/step - loss: 18.1160 - categorical_accuracy: 0.2

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

In [40]:
## Make Predictions ##

In [41]:
res = model.predict(x_test)

1/1 [==============================] - 0s 422ms/step


In [42]:
actions[np.argmax(res[0])]

'hello'

In [43]:
actions[np.argmax(y_test[0])] # performing good

'hello'

In [44]:
## Save Weights ##

In [45]:
model.save('action.h5')

/Users/yaraayman/Documents/ML projects/Action Detection Project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [46]:
model.save('actions.keras')

In [47]:
## Evaluation Using Confusion Matrix and Accuracy ##

In [48]:
# help us to evaluate what has been detected as true positive and true negative or false ....

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [49]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 15ms/step


In [50]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [59]:
# shape 2*2 -> we want numbers to be in top left corner or bottom right corner
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 1],
        [1, 1]],

       [[4, 0],
        [0, 1]],

       [[2, 1],
        [1, 1]]])

In [58]:
accuracy_score(ytrue, yhat) # if 1.0 then 100% accuracy in our test set (our set was small though)

0.6

In [53]:
## Test in Real Life ##

In [56]:
#1. New detection variables
sequence = [] # collect our 30 frames to predict
sentence = [] # concatenate history of detection
threshold = 0.4 # confidence matrix

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        #Draw landmarks
        draw_styled_landmarks(image, results)

        #2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        #sequence.append(keypoints)
        sequence = sequence[:30]  # last 30 frames

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

        # 3. Visualization Logic
        if res[np.argmax(res)] > threshold: # highest score result > threshold
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]: # checking if the word matches the last prediction
                    sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)]) # if no sentence

        if len(sentence) > 5:
            sentence = sentence[-5:] # grabbing last 5 values

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (225, 225, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image) #img from webcam

        # Breaking gracefully by qlicking q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

IndexError: index 10 is out of bounds for axis 0 with size 5

In [61]:
    cap.release()
    cv2.destroyAllWindows()

In [64]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))


        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:

                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti